사용한 데이터 고서 한자 인식 (OCR) 데이터
(해서체 데이터 TS 1)

In [1]:
import numpy as np
import pandas as pd
import os
import random
from tqdm import tqdm
from PIL import Image, ImageDraw
import pickle

import json

Annotation format을 YOLO format으로 변경

In [ ]:
list1 = os.listdir('1.Training/라벨링데이터/TL1/')
list2 = os.listdir('1.Training/원천데이터/TS1/')
path = '1.Training/라벨링데이터/TL1/'
path2 = '1.Training/원천데이터/TS1/'

In [ ]:
a_json = open(path+list1[0], encoding = 'utf-8')
a_dict = json.load(a_json)

In [ ]:
a_dict['Image_Text_Coord'][0][0]['bbox'][1] * 0.9

In [ ]:
# 한자 인덱스 사전 만들기

dic = {}
cnt = 0
for i in range(10000):
    a_json = open(path+list1[i], encoding = 'utf-8')
    a_dict = json.load(a_json)
    for j in range(len(a_dict['Image_Text_Coord'])):
        for k in range(len(a_dict['Image_Text_Coord'][j])):
            hanza = a_dict['Image_Text_Coord'][j][k]['label']
            if hanza not in dic :
                dic[hanza] = cnt
                cnt += 1

전처리

In [ ]:
# 사용할 데이터의 상하단 여백 자름


for idx in range(10000) :   # 깨끗한 해서체 데이터 중 10000개의 상하단 여백을 자름
    img = Image.open(path2+list2[idx+1])
    a_json = open(path+list1[idx+1], encoding = 'utf-8')
    a_dict = json.load(a_json)

    height = a_dict['Image_Height']
    width = a_dict['Image_Width']

    x = 0

    min_box = []
    max_box = []
    for i in range(len(a_dict['Image_Text_Coord'])):
        min_box.append(a_dict['Image_Text_Coord'][i][0]['bbox'][1])
    min_y = min(min_box)

    for i in range(len(a_dict['Image_Text_Coord'])):
        max_box.append(a_dict['Image_Text_Coord'][i][-1]['bbox'][1] + a_dict['Image_Text_Coord'][i][-1]['bbox'][3])
    max_y = max(max_box)

    if max_y < 2800 :
        max_y = 2800

    y = min_y * 0.9
    y2 = (height - max_y) * 0.9
    height = y + (height - y - y2)

    img2 = img.crop((x,y,width,height))
    img2.save('crop/{}.jpg'.format(idx+1))


    with open("crop/{}.txt".format(idx+1), "w") as f:  
        for i in a_dict['Image_Text_Coord'] :
            for j in i :
                xx = j['bbox'][0]
                yy = j['bbox'][1] - y
                ww = j['bbox'][2]
                hh = j['bbox'][3]
                f.write('{} {} {} {} {}\n'.format(dic[j['label']], xx , yy, ww, hh))

In [ ]:
# resize와 padding을 수행하는 함수

import cv2


def padding(img, set_size):
    if len(img.shape) < 3 :
        img = np.expand_dims(img,axis=2)

    try:
        h,w,c = img.shape
    except:
        print('파일을 확인후 다시 시작하세요.')
        raise

    if h < w:
        new_width = set_size
        new_height = int(new_width * (h/w))
    else:
        new_height = set_size
        new_width = int(new_height * (w/h))

    if max(h, w) < set_size:
        img = cv2.resize(img, (new_width, new_height), cv2.INTER_CUBIC)
    else:
        img = cv2.resize(img, (new_width, new_height), cv2.INTER_AREA)

 
    try:
        h,w,c = img.shape
    except:
        print('파일을 확인후 다시 시작하세요.')
        raise

    delta_w = set_size - w
    delta_h = set_size - h
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    img = Image.fromarray(new_img)

    return img

In [ ]:
#이미지 Resize 및 Annotation 데이터 수정 후 저장

for idx in range(1,10001): #1제이슨 2이미지
    txt_path = 'crop/{}.txt'.format(idx)
    f = open(txt_path, 'r')
    lines = f.readlines()
    txt_ls = []
    for line in lines:
        line = line.strip().split(' ')
        txt_ls.append(line)
    f.close()

    img,hh,ww = padding('{}.jpg'.format(idx))
    sv_path = 'crop_resize800/{}.jpg'.format(idx)


    wi = ww
    he = hh

    with open("crop_resize800/{}.txt".format(idx), "w") as f:  
        for i in range(len(txt_ls)):
            cl = int(txt_ls[i][0])
            xx = float(txt_ls[i][1])
            yy = float(txt_ls[i][2])
            ww = float(txt_ls[i][3])
            hh = float(txt_ls[i][4])




            x = (((xx+(he-wi)/2)*2 + ww)/2) / he
            y = ((yy*2 + hh)/2) / he
            w = ww / he
            h = hh / he   
            f.write('{} {} {} {} {}\n'.format(cl, x , y, w, h))
        
        # for i in a_dict['Image_Text_Coord'] :
        #     for j in i :
        #         x = (((j['bbox'][0]+(he-wi)/2)*2 + j['bbox'][2])/2) / he
        #         y = ((j['bbox'][1]*2 + j['bbox'][3])/2) / he
        #         w = j['bbox'][2] / he
        #         h = j['bbox'][3] / he
        #         f.write('{} {} {} {} {}\n'.format(dic[j['label']], x , y, w, h))

    img = img.resize((800,800))
    img.save(sv_path)

In [ ]:
f = open("crop_resize800/1.txt", 'r')
# f = open("BoundingBox/000000000005866_003.txt", 'r')
line = f.readlines()
f.close()

In [ ]:
# 샘플 하나 바운딩박스 확인

img = Image.open('crop_resize800/1.jpg').convert('RGB')
# img = Image.open('1.Training/원천데이터/TS1/000000000005866_003.jpg').convert('RGB')
# img.show()

img2 = np.array(img)


x_list = []
y_list = []
max_X_list = []
max_y_list = []
draw = ImageDraw.Draw(img)

for li in line :
    split_data = li.split(' ')
    list22 = []
    for i in range(1,5):
        if i < 4 :
            list22.append(float(split_data[i]))
        else :
            list22.append(float(split_data[i][:-1]))

    x = list22[0] * img2.shape[1] - (list22[2] * img2.shape[1]) / 2
    y = list22[1] * img2.shape[0] - (list22[3] * img2.shape[0]) / 2
    max_x = list22[0] * img2.shape[1] + (list22[2] * img2.shape[1]) / 2
    max_y = list22[1] * img2.shape[0] + (list22[3] * img2.shape[0]) / 2
    
    draw.rectangle((x,y,max_x,max_y), outline=(0,255,0), width = 3)

img.show()

Train / Valid 분리

In [7]:
index_list = [i for i in range(10000)]

random.shuffle(index_list)

train_index = index_list[:8000]
valid_index = index_list[8000:]

len(train_index), len(valid_index)

(8000, 2000)

In [ ]:
path = 'crop_resize800/'

#train, valid 경로를 담은 text 파일 생성

f = open('train.txt', 'w')
for index in train_index :
    tr = path+index
    if tr[-3:] == 'jpg':
        # 문자 입력
        f.write(path + tr+"\n")
f.close()

f = open('valid.txt', 'w')
for index in valid_index :
    va = path+index
    if va[-3:] == 'jpg':
        # 문자 입력
        f.write(path + va+"\n")
f.close()

In [ ]:
f = open('ClassNames.names', 'w', encoding='utf-8')


for i in dic.keys() :
    f.write(i+'\n')

# 파일 닫기
f.close()